<a href="https://colab.research.google.com/github/Michael-Jimenez-C/CieCom2/blob/main/proyecto_CII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sympy as sp

In [46]:
class Cuarto:
  def __init__(self,capacitancia,temperatura,fuente=0,Rfuente=1):
    t=sp.symbols("t")
    self.temperatura=temperatura
    self.adyacentes=[]#[(nodo,resistencia)]
    self.capacitancia=capacitancia
    self.fuente=fuente
    self.Rfuente=Rfuente
  def eq(self):
    equ=0
    if type(self.fuente)!=list:
      equ+=(self.fuente-self.temperatura if self.fuente!=0 else 0)/self.Rfuente
    else:
      for i,j in zip(self.fuente,self.Rfuente):
        equ+=(i-self.temperatura if i!=0 else 0)/j
    for i in self.adyacentes:
      equ+=(i[0].temperatura-self.temperatura)*sp.Rational(1,i[1])
    return sp.simplify(equ*sp.Rational(1,self.capacitancia))

In [50]:
t,s=sp.symbols("t s")
T1,T2,T3=sp.symbols("T_{1} T_{2} T_{3}")
A1=Cuarto(10,T1,sp.symbols("T_{i1}"))
B1=Cuarto(5,T2,0)
C1=Cuarto(100000,T3,0)

A1.adyacentes=[(B1,15)]
B1.adyacentes=[(A1,15),(C1,20)]
C1.adyacentes=[(A1,20)]

nodos=[A1,B1,C1]

In [48]:
nodos[1].eq()

T_{1}/75 - 7*T_{2}/300 + T_{3}/100

In [71]:

variables=sp.symbols("T_{1} T_{2} T_{3}")
entradas=[sp.symbols("T_{i1}")]
A=[]
B=[]
for i in range(len(nodos)):
  A.append([])
  B.append([])
  eq=nodos[i].eq()
  for j in range(len(variables)):
    A[-1].append(eq.coeff(variables[j]))
  for j in range(len(entradas)):
    B[-1].append(eq.coeff(entradas[j]))

A=sp.Matrix(A)
B=sp.Matrix(B)
A,B

(Matrix([
 [    -8/75,  1/150,          0],
 [     1/75, -7/300,      1/100],
 [1/2000000,      0, -1/2000000]]), Matrix([
 [1/10],
 [   0],
 [   0]]))

In [54]:
class Edificio:
  pass

r

In [70]:
t,s=sp.symbols("t s")

A=sp.Matrix([[1,1],[0,-1]])
B=sp.Matrix([[0],[1]])
x0=sp.Matrix([[0],[1]])
u=sp.Matrix([[sp.Heaviside(t)*t]])

In [56]:
def solucion(A,B,x0,u):
  t,s=sp.symbols("t s")
  U=sp.Matrix([i[0] for i in sp.laplace_transform(u,t,s)])
  Y=(s*sp.eye(A.shape[0])-A).inv()@(x0+B*U)
  sol=sp.inverse_laplace_transform(Y,s,t)
  return sol


In [73]:
sol=solucion(A,B,sp.Matrix([[5],[5],[5]]),sp.Matrix([[1]]))

<ipython-input-56-511a5afe3ec7>:3: SymPyDeprecationWarning: 

Calling laplace_transform() on a Matrix with noconds=False (the default) is
deprecated. Either noconds=True or use legacy_matrix=False to get the new
behavior.

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-laplace-transform-matrix
for details.

This has been deprecated since SymPy version 1.9. It
will be removed in a future version of SymPy.

  U=sp.Matrix([i[0] for i in sp.laplace_transform(u,t,s)])


KeyboardInterrupt: ignored

In [ ]:
sp.plot(sol[0],0,100)